In [364]:
from threading import Thread, Condition, Lock
from time import sleep, time
from IPython.display import clear_output
import random
import copy

In [5]:
class Environment(object):
    #multiplying by this wil be equivalen to (%): [0, 25, 45, 60, 70, 75, 78, 80, 80]
    _concurrent_loss= [1, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2]

    _audience = [
        [2,   1.3,    1],
        [1.3, 0.9, 0.75],
        [1,  0.75, 0.47]
    ]

    #0 will due to a total score of 0 to this match so will be discarded
    _hour_coef = [
        [  0, 0.55, 0.45,   0],
        [  0,  0.7, 0.75,   0],
        [  0,  0.8,  0.5,   0],
        [0.4,    1,    1, 0.4]
    ]

    _categories = ['A', 'B', 'C']
    _days = ['V', 'S', 'D', 'L']
    _hours = [12, 16, 18, 20]

    _idx_days = range(len(_days))
    _idx_hours = range(len(_hours))


class Match(object):
    def __init__(self, idx_homeCat, idx_awayCat, idx_day, idx_hour):
        self._hCat = idx_homeCat
        self._aCat = idx_awayCat
        self._d = idx_day
        self._h = idx_hour

    def get_raw_time(self):
        return [self._d, self._h]

    def get_away_category(self):
        return Environment._categories[self._aCat]

    def get_home_category(self):
        return Environment._categories[self._hCat]

    def get_base(self):
        return Environment._audience[self._hCat][self._aCat]

    def get_hour_coef(self):
        return Environment._hour_coef[self._h][self._d]

    def get_score(self):
        return self.get_base() * self.get_hour_coef()


class GenotypeTimetable(object):
    def __init__(self, num_matches=20, chain=None):
        '''
        Genotype representation, list of lists: [[1, 2]] ([[D, H], [D, H]])
        where in pos 0 is the index of day, and in pos 1 the index of hour
        '''
        if chain != None:
            self._chain = chain
        else:
            self._chain = []
            for i in range(num_matches):
                self._chain.append([random.choice(Environment._idx_days), random.choice(Environment._idx_hours)])


    def get_applciable_chain(self):
        l = []
        for (day, hour) in self._chain:
            l.append([Environment._days[day], Environment._hours[hour]])

        return l


    def mutate(self, partial=False):
        chrom = random.choice(range(len(self._chain)))

        if partial: #Mutate only one part of the chromosome
            idx_chrom = random.choice([0, 1]) #only this two possible indexes
            if idx_chrom == 0: #Days index
                self._chain[chrom][idx_chrom] = random.choice(Environment._idx_days) #Mutate the day
            else:
                self._chain[chrom][idx_chrom] = random.choice(Environment._idx_hours) #Mutate the hour
        else:
            self._chain[chrom] = [random.choice(Environment._idx_days), random.choice(Environment._idx_hours)] #Mutate the full chromosome


    def cross(self, genotype):
        g1 = copy.deepcopy(self._chain)
        g2 = copy.deepcopy(genotype._chain)

        sl = random.choice(range(len(g1)))
        first = random.choice([0, 1])

        g3 = []
        if first == 0:
            g3 = g1[:sl] + g2[sl:]
        else:
            g3 = g2[:sl] + g1[sl:]

        return GenotypeTimetable(chain=g3)
        

    def count(self, chrom):
        return self._chain.count(chrom)

    def __str__(self):
        return '{}'.format(self.get_applciable_chain())

    def __repr__(self):
        return '{}'.format(self.get_applciable_chain())

    def __getitem__(self, idx):
        return self._chain[idx]

    def __len__(self):
        return len(self._chain)



class FenotypeTimetable(object):
    def __init__(self, matchCombinations):
        '''
        :param matchCombinations: List of lists of combinations of matches (e.g: [[A, A], [A, B]])
        '''
        self._suitable = True
        self._m_comb = []
        for (cat1, cat2) in matchCombinations:
            self._m_comb.append([Environment._categories.index(cat1), Environment._categories.index(cat2)])


    def set_genotype(self, genotype):
        self._gen = genotype
        self._matches = []
        for i, (cat1, cat2) in enumerate(self._m_comb):
            (day, hour) = genotype[i]
            self._matches.append(Match(cat1, cat2, day, hour))


    def get_score(self):
        self._suitable = True
        acum = 0
        for match in self._matches:
            raw_time = match.get_raw_time()
            coincidences = self._gen.count(raw_time) - 1
            coin_coef = Environment._concurrent_loss[coincidences]
            sc = match.get_score()
            acum += sc * coin_coef
            if sc == 0:
                self._suitable = False

        return acum

    def is_suitable(self):
        return self._suitable




gen1 = GenotypeTimetable(5)
gen2 = GenotypeTimetable(5)
print('Gen 1:', gen1)
print('Gen 2:', gen2)

print()
gen3 = gen1.cross(gen2)
print('Gen 1 + 2 (3)   :', gen3)
gen3.mutate(partial=False)
print('Gen 3 (mutation):', gen3)
print()
fen = FenotypeTimetable([['A', 'A'], ['B', 'A'], ['A', 'C'], ['A', 'B'], ['C', 'C']])
fen.set_genotype(gen3)
match1 = fen._matches[0]
print('Match 1 info: ', match1.get_home_category(), match1.get_away_category(), match1.get_base(), match1.get_hour_coef(), match1.get_score())
print('Match scores: ', [i.get_score() for i in fen._matches])
print('Total score (fitness): ', fen.get_score())

Gen 1: [['S', 16], ['D', 18], ['D', 20], ['L', 18], ['S', 16]]
Gen 2: [['V', 18], ['V', 20], ['D', 16], ['V', 16], ['V', 20]]

Gen 1 + 2 (3)   : [['S', 16], ['D', 18], ['D', 20], ['V', 16], ['V', 20]]
Gen 3 (mutation): [['S', 16], ['D', 18], ['S', 20], ['V', 16], ['V', 20]]

Match 1 info:  A A 2 0.7 1.4
Match scores:  [1.4, 0.65, 1, 0.0, 0.188]
Total score (fitness):  3.238


In [397]:
class God(object):
    __shared = {
        'COND': Condition(),
        'STOP': False,
        'DONES': 0,

        'genotypes': [],
        'fenotypes': [],

        'score_list': [],
        'suitable_list': [],
    }

    def __init__(self, matches, living_things=8, iterations=50, reproductions=1, mutations=1):
        self.MATCHES = matches

        self.__living_things = living_things
        self.__iterations = iterations
        self.__reproductions = reproductions
        self.__mutations = mutations

        self.fitness_list = []

        self.cross_genotypes = []
        self.cross_fenotypes = []

        self.best_genotype = None
        self.best_score = -1


    @staticmethod
    def living_thing(shared_index):
        cond = God.__shared['COND']
        while God.__shared['STOP'] == False:
            gen = God.__shared['genotypes'][shared_index]
            fen = God.__shared['fenotypes'][shared_index]
            fen.set_genotype(gen)

            God.__shared['score_list'][shared_index] = fen.get_score()
            God.__shared['suitable_list'][shared_index] = fen.is_suitable()
            with cond:
                God.__shared['DONES'] += 1 #atomic
                cond.wait()


    def wait_until_all_complete(self):
        while God.__shared['DONES'] != self.__living_things:
            pass


    def notify(self):
        cond = God.__shared['COND']
        with cond:
            God.__shared['DONES'] = 0
            cond.notifyAll()


    def start_threads(self):
        God.__shared['STOP'] = False
        l = len(self.MATCHES)
        for i in range(self.__living_things):
            gen = GenotypeTimetable(l)
            fen = FenotypeTimetable(self.MATCHES)
            God.__shared['genotypes'].append(gen)
            God.__shared['fenotypes'].append(fen)
            God.__shared['score_list'].append(-1)
            God.__shared['suitable_list'].append(False)
            Thread(target=God.living_thing, args=(i,)).start()


    def fitness(self):
        score_list = God.__shared['score_list']
        total = sum(score_list)
        self.fitness_list.clear()
        for score in score_list:
            perc = int(score / total * 100)
            self.fitness_list.append(perc)


    def fitness_loss(self):
        score_list = God.__shared['score_list']
        total = sum(score_list)
        inv = []
        for score in score_list:
            inv.append(total - score)

        total = sum(inv)
        self.fitness_list.clear()
        for score in inv:
            perc = int(score / total * 100)
            self.fitness_list.append(perc)


    def selection_cross(self):
        l = []
        for i in range(self.__living_things):
            l += [i] * self.fitness_list[i]

        genotypes = God.__shared['genotypes']
        fenotypes = God.__shared['fenotypes']
        for x in range(self.__reproductions):
            gen = genotypes[random.choice(l)].cross(genotypes[random.choice(l)])
            fen = FenotypeTimetable(self.MATCHES)
            self.cross_genotypes.append(gen)
            self.cross_fenotypes.append(fen)


    def mutation(self):
        genotypes = God.__shared['genotypes']
        for gen in genotypes:
            gen.mutate()

        for gen in self.cross_genotypes:
            gen.mutate()


    def eval_reselect(self):
        score_list = God.__shared['score_list']
        genotypes = God.__shared['genotypes']
        fenotypes = God.__shared['fenotypes']
        for (gen, fen) in zip(self.cross_genotypes, self.cross_fenotypes):
            fen.set_genotype(gen)
            score_list.append(fen.get_score())
            genotypes.append(gen)
            fenotypes.append(fen)

        self.fitness_loss()
        for x in range(len(self.cross_genotypes)):
            l = []
            for i, fit in enumerate(self.fitness_list):
                l += [i] * (100 - fit) #higher the better

            idx = random.choice(l)
            del self.fitness_list[idx]
            del score_list[idx]
            del genotypes[idx]
            del fenotypes[idx]

        self.cross_fenotypes.clear()
        self.cross_genotypes.clear()

    
    def select_solution(self):
        suitable_list = God.__shared['suitable_list']
        genotypes = God.__shared['genotypes']
        score_list = God.__shared['score_list']
        for (suitable, gen, score) in zip(suitable_list, genotypes, score_list):
            if suitable and score > self.best_score:
                self.best_score = score
                self.best_genotype = gen


    def start_life(self):
        God.__shared['STOP'] = False
        self.start_threads() #initiate threads
        self.wait_until_all_complete()

        self.fitness()


        #loop
        progress_size = 50
        print('[>', end='')
        _s = 0
        _p = 0
        step = self.__iterations / 50
        for i in range(self.__iterations):
            self.select_solution()

            self.selection_cross()
            self.mutation()

            self.notify()
            self.wait_until_all_complete()
            self.eval_reselect()
            if _s >= step:
                _p += 1
                _s = 0
                clear_output(wait=True)
                print('[{}>{}] {:.2f}%'.format(''.join(['=']*_p), ''.join([' ']*(progress_size-_p)), _p/progress_size*100),  end='')

            _s += 1

        self.notify()
        God.__shared['STOP'] = True
        _p += 1
        clear_output(wait=True)
        print('[{}>{}] {:.2f}%'.format(''.join(['=']*_p), ''.join([' ']*(progress_size-_p)), _p/progress_size*100),  end='')
        print(' DONE!')


god = God([['B', 'A'], ['B', 'A'], ['C', 'C'], ['B', 'A'], ['C', 'C'], ['B', 'C'], ['B', 'B'], ['B', 'B'], ['B', 'C'], ['A', 'B']], iterations=100, living_things=20, mutations=1, reproductions=2)

t = time()
god.start_life()
print('Iterations: 100')
print('Population: 20')
print('Mutations: 1')
print('Reproductions: 2')
print('Best score: ', god.best_score)
print('Best genotype: ', god.best_genotype)
print('Elapsed: ', (time() - t) / 60, 'mins')

[==================================================>] 100.00% DONE!
Iterations: 100
Population: 20
Mutations: 1
Reproductions: 2
Best score:  5.864625
Best genotype:  [['D', 20], ['D', 12], ['S', 20], ['L', 20], ['D', 12], ['S', 20], ['L', 16], ['D', 16], ['S', 18], ['V', 12]]
Elapsed:  0.05636890331904094 mins


In [398]:
god = God([['B', 'A'], ['B', 'A'], ['C', 'C'], ['B', 'A'], ['C', 'C'], ['B', 'C'], ['B', 'B'], ['B', 'B'], ['B', 'C'], ['A', 'B']], iterations=10000, living_things=20, mutations=1, reproductions=2)

t = time()
god.start_life()
print('Iterations: 10000')
print('Population: 20')
print('Mutations: 1')
print('Reproductions: 2')
print('Best score: ', god.best_score)
print('Best genotype: ', god.best_genotype)
print('Elapsed: ', (time() - t) / 60, 'mins')

[==================================================>] 100.00% DONE!
Iterations: 10000
Population: 20
Mutations: 1
Reproductions: 2
Best score:  6.505000000000001
Best genotype:  [['S', 16], ['S', 20], ['L', 20], ['L', 16], ['V', 12], ['D', 20], ['S', 20], ['L', 20], ['D', 18], ['S', 12]]
Elapsed:  6.006570935249329 mins
